Gutes Notebook hier: https://www.kaggle.com/code/vatsalmavani/music-recommendation-system-using-spotify-dataset/notebook#Building-Music-Recommendation-System-using-Spotify-Dataset

In [1]:
# Import dependencies
import pandas as pd
import pickle

import numpy as np
import matplotlib.pyplot as plt
import plotly.offline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score

In [3]:
# Load data
## Download here https://www.kaggle.com/datasets/gauthamvijayaraj/spotify-tracks-dataset-updated-every-week
## And put into ../data folder
data = pd.read_csv("../data/spotify_tracks.csv")

#data = pd.read_csv("../data/data_for_recommender.csv")


# Feature Names
# track_id,track_name,artist_name,year,popularity,artwork_url,album_name,acousticness,danceability,
# duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,
# valence,track_url,language


# Overfitting via #parameter = #features * #datenpunkte 
data = data[(data.popularity > 60) & (data.language == 'English')]

# TODOs:
# - [-] Mehrere vorgefertigte Playlists als Datengrundlage (pro Genre)
# - Genre auswählen als Präferenz
# - [-] Playlist von denen hochladbar machen; (In Präsentation ggf nur erwähnen und nicht live demonstrieren)
# - Nach Auswahl der Playlist deren Tracks und die in unserem Datenset, die aus der recommendation rausgekommen sind als 1 markieren
# 


In [4]:
# Filter X

#col_names = ['track_name', 'artist_name', 'instrumentalness', 'danceability']
col_names = ['track_name', 'artist_name', 'tempo', 'danceability']
val_col_names = col_names[2:]
n_samples = 500

df_X = data[col_names][:n_samples]
X = df_X.to_numpy()
# X_track_names = X[:,0]
# X_artist_names = X[:,1]
X_1 = X[:,2]
X_2 = X[:,3]

# X_1 = X[:,0]
# X_2 = X[:,1]

In [5]:
# Create y
y = np.zeros(shape=X_1.shape[0]) 
y[np.logical_or(X_1 > 120, X_2 > 0.6)] = 1
print(f'Num likes:{int(np.sum(y))} / {y.shape[0]}')


Num likes:390 / 500


In [6]:
# # Preprocess data
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# Apply noise
# noise_factor = 0.05
# noise_idx = np.random.choice(len(y), size=int(noise_factor*len(y)), replace=False)
# y[noise_idx] = 1 - y[noise_idx]
# print(f'Num likes:{int(np.sum(y))} / {y.shape[0]}')

# NOTE Wenn hier X gemischt wird, darf ich nicht nur die Data columns nehmen, sonst werden artist und track namen losgelöst
# Split the data

# Train only on value columns
X_train, X_test, y_train, y_test = train_test_split(df_X[val_col_names], y, test_size=0.95, random_state=42)
X_train2, _, y_train2, _ = train_test_split(df_X[val_col_names], y, test_size=0.2, random_state=42)

In [7]:
# Train a Support Vector Machine classifier
clf_optimal = SVC(kernel='rbf', C=10000,  random_state=42)
clf_optimal.fit(X_train2, y_train2)

# Overfit
clf_overfit = SVC(kernel='rbf', C=100000000, gamma='scale', random_state=42)
clf_overfit.fit(X_train, y_train)

# Underfit
clf_underfit = SVC(kernel='rbf', C=1, random_state=42)
clf_underfit.fit(X_train, y_train)

SVC(C=1, random_state=42)

In [8]:
x_feature_1=df_X[val_col_names[0]]
x_feature_2=df_X[val_col_names[1]]

In [ ]:
## Plot Prediction
import plotly.graph_objects as go
import plotly.express as px

def plot_pred(classifier, X, y, title:str):
    # Create a mesh grid
    # x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
    # y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
    x_min, x_max = X[:, 0].min(), X[:, 0].max()
    y_min, y_max = X[:, 1].min(), X[:, 1].max()
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                         np.arange(y_min, y_max, 0.02))
    # xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
    #                    np.linspace(y_min, y_max, 100))

    # Predict on the mesh grid
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the decision boundary
    plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.RdYlBu)
    # plt.contourf(xx, yy, Z, alpha=0.3)

    # Plot the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu, edgecolor='black')
    # plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', marker='o', s=20)
    
    plt.title(title)
    plt.xlabel('Instrumentalness')
    plt.ylabel('Danceability')

# TODO Zusätzlich training data plotten (?)
# def px_plot_pred(classifier, X, y, title:str):
def px_plot_pred(classifier, X_train, X_test, y_train, y_test, title:str):
    # Create a mesh grid
    # x_min, x_max = X_scaled[:, 0].min() - 1, X_scaled[:, 0].max() + 1
    # y_min, y_max = X_scaled[:, 1].min() - 1, X_scaled[:, 1].max() + 1
    x_min, x_max = x_feature_1.min(), x_feature_1.max()
    y_min, y_max = x_feature_2.min(), x_feature_2.max()
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))

    # Predict on the mesh grid
    Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the training points
    ## pd.DataFrame(data=X, columns=col_names)
    df = pd.DataFrame(X_test.copy())
    df['prediction'] = y_test # classifier.predict(X_test)
    df['track_name'] = df_X.loc[X_test.index, 'track_name']
    df['artist_name'] = df_X.loc[X_test.index, 'artist_name']

    # TODO Where to plot classifier.predict(X_test)?
    fig = px.scatter(df, x=df[val_col_names[0]], y=df[val_col_names[1]], color=df['prediction'],
                     hover_data={'tempo':False,
                                 'danceability':False,
                                 'prediction':False,
                                 'artist_name': True,
                                 'track_name':True},
                     color_continuous_scale=['red', 'green'])
    fig.update_coloraxes(showscale=False)
    fig.update_traces(marker=dict(line=dict(color='black', width=1)))

    fig.add_contour(x=xx[0], y=yy[:, 0], z=Z,
                    colorscale=['red', 'green'],  #'YlGn'
                    ncontours=2,
                    opacity=0.8,
                    hoverinfo='skip',
                    showscale=False)
    fig.update_layout(
      #title=title,
      width=600,
      height=600,
      xaxis_title='Instrumentalness',
      yaxis_title='Danceability',
    )
    fig.show()
    # TODO + Legende
    # TODO Scaler
    # TODO (Optional) Hide some figure buttons
    # TODO (IDEA Button zum togglen zwischen Test und Training Daten)

px_plot_pred(clf_optimal, X_train, X_test, y_train, y_test, 'Optimal Classifier')
# px_plot_pred(clf_overfit, X_train, X_test, y_train, y_test, 'Overfit Classifier')
# px_plot_pred(clf_underfit, X_train, X_test, y_train, y_test, 'Underfit Classifier')


In [ ]:
# plot_pred(clf_optimal, X, y, 'Optimal Classifier')
# plt.show()
# plot_pred(clf_overfit, X, y, 'Overfit Classifier')
# plt.show()
# plot_pred(clf_underfit, X, y, 'Underfit Classifier')
# plt.show()
# 
# y_pred_underfit = clf_underfit.predict(X_test)
# print(f'Underfitting Accuracy: {accuracy_score(y_test, y_pred_underfit)}')
# # plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plot_pred(clf_underfit, X_train, y_train, 'Underfitted Train')
# plt.subplot(1, 2, 2)
# plot_pred(clf_underfit, X_test, y_test, 'Underfitted Test')
# plt.show()
# 
# 
# y_pred_optimal = clf_optimal.predict(X_test)
# print(f'Middle Accuracy: {accuracy_score(y_test, y_pred_optimal)}')
# plt.subplot(1, 2, 1)
# plot_pred(clf_optimal, X_train, y_train, 'Middle Train')
# plt.subplot(1, 2, 2)
# plot_pred(clf_optimal, X_test, y_test, 'Middle Test')
# plt.show() # .savefig()
# 
# # Overfitting example: Deep network with too many neurons
# y_pred_overfit = clf_overfit.predict(X_test)
# print(f'Overfitting Accuracy: {accuracy_score(y_test, y_pred_overfit)}')
# plt.subplot(1, 2, 1)
# plot_pred(clf_overfit, X_train, y_train, 'Overfitted Train')
# plt.subplot(1, 2, 2)
# plot_pred(clf_overfit, X_test, y_test, 'Overfitted Test')
# plt.show()

# -----------------------------
# fig = pickle.load(open('../backend/assets/testplot.pkl', 'rb'))
# fig.show()
# fig.to_html()



# Spotify Recommender

In [57]:
from collections import defaultdict
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Load Data
data = pd.read_csv("../data/data_for_recommender.csv")

scaler = StandardScaler()
scaler.fit(data.select_dtypes(np.number))

StandardScaler()

In [58]:
number_cols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
               'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']

def get_song_data(song, spotify_data):
  try:
    song_data = spotify_data[(spotify_data['name'] == song['name'])
                             & (spotify_data['year'] == song['year'])].iloc[0]
    return song_data

  except IndexError:
    return None

def get_mean_vector(song_list, spotify_data):
  song_vectors = []

  for song in song_list:
    song_data = get_song_data(song, spotify_data)
    if song_data is None:
      print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
      continue
    song_vector = song_data[number_cols].values
    song_vectors.append(song_vector)

  song_matrix = np.array(list(song_vectors))
  return np.mean(song_matrix, axis=0)

def flatten_dict_list(dict_list):
  flattened_dict = defaultdict()
  for key in dict_list[0].keys():
    flattened_dict[key] = []

  for dictionary in dict_list:
    for key, value in dictionary.items():
      flattened_dict[key].append(value)

  return flattened_dict


def recommend_songs(song_list, spotify_data, n_songs=10):
  metadata_cols = ['name', 'year', 'artists']
  song_dict = flatten_dict_list(song_list)

  song_center = get_mean_vector(song_list, spotify_data)
  scaled_data = scaler.transform(spotify_data[number_cols])
  scaled_song_center = scaler.transform(song_center.reshape(1, -1))
  distances = cdist(scaled_song_center, scaled_data, 'cosine')
  index = list(np.argsort(distances)[:, :n_songs][0])

  rec_songs = spotify_data.iloc[index]
  rec_songs = rec_songs[~rec_songs['name'].isin(song_dict['name'])]
  return rec_songs[metadata_cols].to_dict(orient='records')

In [60]:
recommend_songs([#{'name': 'Smells Like Teen Spirit', 'year': 1991},
                 #{'name': 'Lithium', 'year': 1992},
                 #{'name': 'All Apologies', 'year': 1993},
                 #{'name': 'Stay Away', 'year': 1993}],
                  {'name': 'Come As You Are', 'year': 1991}], data)

C:\Uni\AI Guardians\ai-guardian\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[{'name': 'In the End', 'year': 2000, 'artists': "['Linkin Park']"},
 {'name': 'Shimmer', 'year': 1997, 'artists': "['Fuel']"},
 {'name': 'Symphony Of Destruction', 'year': 1992, 'artists': "['Megadeth']"},
 {'name': 'Breaking the Habit', 'year': 2003, 'artists': "['Linkin Park']"},
 {'name': 'Lovers Rock', 'year': 2014, 'artists': "['TV Girl']"},
 {'name': 'Fell On Black Days', 'year': 1994, 'artists': "['Soundgarden']"},
 {'name': 'New Divide', 'year': 2009, 'artists': "['Linkin Park']"},
 {'name': 'War of Change',
  'year': 2012,
  'artists': "['Thousand Foot Krutch']"},
 {'name': 'Freak', 'year': 1997, 'artists': "['Silverchair']"}]